In [ ]:
## 6) Structured Cropping
Using a sliding window the images are cropped to be able to be reconstructed later

def readAndCropImages(data_dir, with_labels=True):
    window_shape = (128, 128)
    image_ids = os.listdir(data_dir)

    X = np.zeros((0, window_shape[1], window_shape[0], 3), dtype=np.uint8)
    Y = np.zeros((0, window_shape[1], window_shape[0], 1), dtype=np.bool_)
    
    i = 0
    for _, imgID in enumerate(image_ids):

        image = Image.open(f"{data_dir}/{imgID}/images/{os.listdir(f'{data_dir}/{imgID}/images')[0]}").convert('RGB')
        
        offset_y = 0
        while offset_y < image.size[1]:
            offset_x = 0
            while offset_x < image.size[0]:
                c = tf.image.crop_to_bounding_box(
                    image, offset_y, offset_x,
                    window_shape[1] if offset_y+window_shape[1] <= image.size[1] else image.size[1] - offset_y,
                    window_shape[0] if offset_x+window_shape[0] <= image.size[0] else image.size[0] - offset_x
                )
                X_tmp = np.zeros((window_shape[1], window_shape[0], 3), dtype=np.uint8)
                X_tmp[:c.shape[0],:c.shape[1]] = c
                i += 1
                X = np.append(X, X_tmp).reshape((i, window_shape[1], window_shape[0], 3))
                
                offset_x += window_shape[0]
            offset_y += window_shape[1]
        
        # Mask
        if with_labels:
            i -= math.ceil(image.size[1]/window_shape[1]) * math.ceil(image.size[0]/window_shape[0]) # reset index
            mask = tf.keras.utils.load_img(f"{data_dir}/{imgID}/combined_masks/{os.listdir(f'{data_dir}/{imgID}/combined_masks')[0]}")
            offset_y = 0
            while offset_y < mask.size[1]:
                offset_x = 0
                while offset_x < mask.size[0]:
                    c = tf.image.crop_to_bounding_box(
                        mask, offset_y, offset_x,
                        window_shape[1] if offset_y+window_shape[1] <= image.size[1] else image.size[1] - offset_y,
                        window_shape[0] if offset_x+window_shape[0] <= image.size[0] else image.size[0] - offset_x
                    )[:,:,:1]
                    Y_tmp = np.zeros((window_shape[1], window_shape[0], 1), dtype=np.bool_)
                    Y_tmp[:c.shape[0],:c.shape[1]] = c
                    i += 1
                    Y = np.append(Y, Y_tmp).reshape((i, window_shape[1], window_shape[0], 1))
                    offset_x += window_shape[0]
                offset_y += window_shape[1]

    return X, Y, image_ids


def reconstructPredictions(predictions, data_dir, window_shape=(128,128), image_mode="L"):
    
    image_ids = os.listdir(data_dir)
    
    X = []
    
    i = 0
    for image_i, imgID in enumerate(image_ids):
    
        # reconstruct
        image_size = Image.open(f"{data_dir}/{imgID}/images/{os.listdir(f'{data_dir}/{imgID}/images')[0]}").size
        reconstructed_image = Image.new(image_mode, image_size)

        offset_y = 0
        while offset_y < image_size[1]:
            offset_x = 0
            while offset_x < image_size[0]:
                reconstructed_image.paste(tf.keras.utils.array_to_img(predictions[i]), (offset_x, offset_y))
                i += 1
                offset_x += window_shape[0]
            offset_y += window_shape[1]

        X.append(reconstructed_image)
        
    return X

X, Y, image_ids = readAndCropImages("./data/stage1_train")

for i in range(27, 33):
    display_img(X[i])

for i in range(27, 33):
    display_img(Y[i])

reconstructions_X = reconstructPredictions(X, "./data/stage1_train", image_mode="RGB")
reconstructions_Y = reconstructPredictions(Y, "./data/stage1_train")

reconstructions_X[5]

reconstructions_Y[5]

In [ ]:
def slidingWindowCropping(X, window_shape=(128, 128)):
    new_X = np.zeros((X.shape[1:]), dtype=np.bool_)
    i = 0
    for i, img in enumerate(X):
        offset_y = 0
        while offset_y < img.size[1]:
            offset_x = 0
            while offset_x < img.size[0]:
                c = tf.image.crop_to_bounding_box(
                    img, offset_y, offset_x,
                    window_shape[1] if offset_y+window_shape[1] <= img.size[1] else img.size[1] - offset_y,
                    window_shape[0] if offset_x+window_shape[0] <= img.size[0] else img.size[0] - offset_x
                )[:,:,:1]
                X_tmp = np.zeros((window_shape[1], window_shape[0], 1), dtype=np.bool_)
                X_tmp[:c.shape[0],:c.shape[1]] = c
                i += 1
                X = np.append(X, X_tmp).reshape((i, window_shape[1], window_shape[0], 1))
                offset_x += window_shape[0]
            offset_y += window_shape[1]
    return new_X